
# LeanTrader — XAUUSD Parameter Sweep

This notebook:
- Loads XAUUSD OHLC data from `data/ohlc/XAUUSD_M15.csv`
- Runs a parameter sweep for:
  - **ADX threshold** (14–30)
  - **ATR multiplier** (1.0–2.0)
  - **FVG lookback window** (3–10 bars)
- Backtests each combo
- Computes **Sharpe** and **Max Drawdown**
- Suggests the best combo overall and by session (Asia/London/NY)


In [ ]:

import itertools
from pathlib import Path

import pandas as pd

from leantrader.backtest.engine import backtest
from leantrader.backtest.metrics import max_drawdown, sharpe
from leantrader.data.feeds import get_ohlc_csv, resample_frames
from leantrader.features.microstructure import engineer
from leantrader.sessions.manager import which_session

DATA_DIR = Path.cwd() / "data" / "ohlc"
path = DATA_DIR / "XAUUSD_M15.csv"
if not path.exists():
    raise FileNotFoundError("Missing XAUUSD_M15.csv. Put your data under data/ohlc/")
df = get_ohlc_csv(str(path))
frames = resample_frames(df)
eng = {k: engineer(v) for k,v in frames.items()}


In [ ]:

# Parameter ranges
adx_vals = [14, 18, 20, 25, 30]
atr_mults = [1.0, 1.2, 1.5, 1.8, 2.0]
fvg_windows = [3, 5, 7, 10]

combos = list(itertools.product(adx_vals, atr_mults, fvg_windows))
print("Total combos:", len(combos))


In [ ]:

def backtest_combo(adx_thr, atr_mult, fvg_win):
    # Copy eng frames shallow
    eng2 = {k:v.copy() for k,v in eng.items()}
    # Add thresholds into engineered features (mock logic)
    # In real case, you'd compile DSL with these params
    sigs = pd.DataFrame(index=eng2["M15"].index)
    sigs["go"] = 0
    sigs["side"] = None
    # Fake: go long when ADX>thr, else skip (demo placeholder)
    sigs.loc[eng2["H1"]["adx_14"] > adx_thr, "go"] = 1
    sigs.loc[sigs["go"]==1, "side"] = "long"
    eq = backtest(eng2["M15"], sigs, risk_cfg=None)
    return eq, sigs


In [ ]:

results = []
for (adx_thr, atr_mult, fvg_win) in combos:
    try:
        eq, sigs = backtest_combo(adx_thr, atr_mult, fvg_win)
        if eq is None or len(eq)==0: continue
        s = sharpe(eq)
        mdd = max_drawdown(eq)
        results.append({"adx_thr":adx_thr,"atr_mult":atr_mult,"fvg_win":fvg_win,
                        "sharpe":s,"maxdd":mdd})
    except Exception as e:
        print("Err on", (adx_thr,atr_mult,fvg_win), ":", e)

resdf = pd.DataFrame(results)
resdf.sort_values("sharpe", ascending=False).head(10)


In [ ]:

# Best combo overall
best = resdf.sort_values("sharpe", ascending=False).iloc[0]
print("Best overall params:", best.to_dict())


In [ ]:

# Session-specific analysis
session_metrics = []
for (adx_thr, atr_mult, fvg_win) in combos[:20]:  # top 20 combos for speed
    eq, sigs = backtest_combo(adx_thr, atr_mult, fvg_win)
    if eq is None or len(eq)==0: continue
    df = sigs.copy()
    df["session"] = [which_session(pd.Timestamp(ts)) for ts in df.index]
    for sess in df["session"].unique():
        s_eq = eq[df["session"]==sess]
        if len(s_eq)==0: continue
        session_metrics.append({
            "adx_thr":adx_thr,"atr_mult":atr_mult,"fvg_win":fvg_win,
            "session":sess,
            "sharpe":sharpe(s_eq),
            "maxdd":max_drawdown(s_eq)
        })

sessdf = pd.DataFrame(session_metrics)
sessdf.groupby("session").apply(lambda x: x.sort_values("sharpe",ascending=False).head(3))



## Next steps
- Replace the mock `backtest_combo` logic with a DSL compile using the thresholds.
- Save best combos per session to a YAML (permanent tuned params).
- Integrate into `policy/dispatcher.py` so XAUUSD uses session-optimized params.
